<a href="https://colab.research.google.com/github/miguel10028/IA_UdeA/blob/main/04%20-%20modelo%20con%20preprocesado%20y%20Gradient%20Boosting%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega Final
En este entrega tendrás que someter tu solución a Kaggle. CADA MIEMBRO DE CADA GRUPO TIENE QUE SOMETER SU PROPIA SOLUCIÓN, aunque sea la misma que la del resto de componentes del grupo.

Además del contenido de las entregas anteriores tendrás que añadir a tu repositorio github

- README.md con los nombres, cédulas y progama en el que está inscrito cada participante y con los siguientes enlaces y con el enlace al video de la - entrega 2 y el enlace al video de esta entrega.

- **Video subido a YouTube y enlazado en README.md**

    - Elabora un vídeo de entre 3mins y 4mins.

    - En el video ha de explicar brevemente la aproximación a la solución enviada a Kaggle y ha de mostrar vuestra posición en el ranking compartiendo la pantalla mientras se consulta el leaderboard de Kaggle. Esta posición, por supuesto, puede que no sea la definitiva si hay equipos que envian sus soluciones después.

- 99 - modelo solución.ipynb - Un notebook con tu solución completa, desde cargar train.csv hasta generar el archivo que se envía a Kaggle. Mira las NOTES 05.03 para tener un ejemplo.

- al menos dos notebooks adicionales con aproximaciones distantas a la de la solución, porque usan distintos modelos, o estrategias de limpieza o preprocesado, etc.

Los criterios de evaluación son los siguientes:

- CLARIDAD: los notebooks contienen textos que explican brevemente lo que se está haciendo
- COMPLECIÓN: la entrega contiene todos los notebooks que se describieron justo arriba.
- REPRODUCIBILIDAD: la solución que genera el notebook 99 - modelo solución.ipynb coincide con la enviada a Kaggle.

Al final tu repositorio habrá de tener la siguiente estructura:  

    +-  README-md
    +-  01 - exploración.ipynb
    +-  02 - preprocesado.ipynb
    +-  03 - modelo con preprocesado de tal forma y SVM.ipynb
    +-  04 - modelo con preprocesado de otra forma y Random Forest.ipynb
    +-  99 - modelo solución.ipynb
    
El nombrado de los archivos ES ESTRICTO. Tiene que haber un fichero README.md y los notebooks 01, 02 y 99 nombrados EXACTAMENTE IGUAL que se indica arriba. Los notebooks 03, 04 y posteriores (si tienes más), han de ser nombrados con ese esquema (número consecutivo - nombre_que_escojes.ipynb). Cualquier archivo con otro nombre será ignorado.

Cada miembro del grupo ES RESPONSABLE DE SU PROPIA ENTREGA, independientemente de si sus compañeros de grupo entregan o no. Esto incluye rellenar el formulario y tener un github personal, aunque el contenido sea el mismo que el del resto de los compañeros de grupo.

Si cambias de grupo o te añades a un grupo que ya exista tienes que enviar un correo a raul.ramos@udea.edu.co, felipe.rodrigueza@udea.edu.co informándolo con copia a tus nuevos compañeros indicando las razones del cambio y cuál será tu rol en el nuevo grupo. Tus nuevos compañeros deberán de responder ese correo autorizandote a que formes parte del grupo.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

## Download data directly from Kaggle

- create a file `kaggle.json` with your authentication token (in kaggle $\to$ click user icon on top-right $\to$ settings $\to$ API create new token)
- upload it to this notebook workspace
- run the following cell

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


## unzip and inspect data

In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#Importamos librerias
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,mean_absolute_error, accuracy_score, mean_squared_error, precision_score, f1_score,recall_score
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import itertools
from sklearn.svm import SVC
from sklearn.metrics import recall_score
import re

In [ ]:
# Lectura del archivo "train.csv"

nombre_archivo = "train.csv"

#Creamos el DataFrame
data = pd.read_csv(nombre_archivo)

# Mostrar un breve resumen del archivo "train.csv"
data.head(10)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Secundaria (Bachillerato) completa,No,medio-alto
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Técnica o tecnológica completa,Si,alto
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,Si,medio-bajo
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Educación profesional incompleta,Si,medio-bajo
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Postgrado,Si,alto


In [ ]:
#Información dimensiones archivo "train.csv":
print("\nTamaño del archivo:",
      "\nFilas:", data.shape[0],
      "\nColumnas:", data.shape[1])

#Verificar datos faltantes para la limpieza y el preprocesado
print("\nDatos Faltantes por columna:\n",data.isnull().sum())


Tamaño del archivo: 
Filas: 692500 
Columnas: 12

Datos Faltantes por columna:
 ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [ ]:
df = data.copy()

# Obtenemos la Moda para rellenar datos faltantes ya que no son valores numéricos
columnas_con_datos_faltantes = ['ESTU_VALORMATRICULAUNIVERSIDAD','ESTU_PAGOMATRICULAPROPIO','ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA','FAMI_TIENEINTERNET','FAMI_EDUCACIONPADRE','FAMI_EDUCACIONMADRE']
moda_columnas_especificas = data[columnas_con_datos_faltantes].mode()
print(moda_columnas_especificas)

#Remplazamos valores NaN
df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(df['ESTU_VALORMATRICULAUNIVERSIDAD'].mode()[0])
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna(df['ESTU_PAGOMATRICULAPROPIO'].mode()[0])
df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(df['ESTU_HORASSEMANATRABAJA'].mode()[0])
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna(df['FAMI_ESTRATOVIVIENDA'].mode()[0])
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna(df['FAMI_TIENEINTERNET'].mode()[0])
df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna(df['FAMI_EDUCACIONPADRE'].mode()[0])
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna(df['FAMI_EDUCACIONMADRE'].mode()[0])



           ESTU_VALORMATRICULAUNIVERSIDAD ESTU_PAGOMATRICULAPROPIO  \
0  Entre 1 millón y menos de 2.5 millones                       No   

  ESTU_HORASSEMANATRABAJA FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET  \
0         Más de 30 horas            Estrato 2                 Si   

                  FAMI_EDUCACIONPADRE                 FAMI_EDUCACIONMADRE  
0  Secundaria (Bachillerato) completa  Secundaria (Bachillerato) completa  


In [ ]:
#Verificar que ya no hay datos faltantes
print("\nDatos Faltantes por columna:\n",df.isnull().sum())


Datos Faltantes por columna:
 ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


# Justificación
La asignación de valores normalizados entre 0 y 1 para los rangos de matrícula sigue un enfoque lineal, en el que cada categoría se distribuye de manera equitativa a lo largo del intervalo [0, 1]. La lógica detrás de estos valores se basa en:

- **Rango más bajo (500 mil a menos de 1 millón)**: Asignado el valor 0 porque es el valor mínimo en la escala.

- **Rango más alto (5.5 millones a menos de 7 millones)**: Asignado el valor 1 porque representa el valor más alto.

- **Rangos intermedios**: Se asignan valores intermedios (0.25, 0.50, 0.75) proporcionalmente, basados en la distancia entre los valores mínimos y máximos, usando una distribución equitativa. Con incrementos de 0.25, cada paso entre los rangos es igual, lo que puede ser más simple de interpretar. En este caso, todos los rangos tienen una distancia fija entre sí (0.25), lo cual puede ser útil para simplificar el análisis posterior o cuando se desea más consistencia entre los valores.

**Valores asignados**:

- **Entre 500 mil y menos de 1 millón**: 0.0

- **Entre 1 millón y menos de 2.5 millones**: 0.25

- **Entre 2.5 millones y menos de 4 millones**: 0.5

- **Entre 4 millones y menos de 5.5 millones**: 0.75

- **Entre 5.5 millones y menos de 7 millones**: 1.0

In [ ]:
# Crear un diccionario para convertir los rangos a valores numéricos
valores_matricula_normalizados = {
    'Entre 500 mil y menos de 1 millón': 0,
    'Entre 1 millón y menos de 2.5 millones': 0.25,
    'Entre 2.5 millones y menos de 4 millones': 0.50,
    'Entre 4 millones y menos de 5.5 millones': 0.75,
    'Entre 5.5 millones y menos de 7 millones': 1
}

# Aplicar el diccionario para convertir la columna en valores numéricos
df['VALOR_MATRICULA_NORMALIZADO'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(valores_matricula_normalizados)

# Normalizar los NaN a un valor dentro del rango 0-1, por ejemplo 0.5
df['VALOR_MATRICULA_NORMALIZADO'] = df['VALOR_MATRICULA_NORMALIZADO'].fillna(0.5)

# Eliminar la columna original
df = df.drop('ESTU_VALORMATRICULAUNIVERSIDAD', axis=1)

# Justificación

Para la columna "ESTU_HORASSEMANATRABAJA" se utilizó la siguiente metodología:

Asignar valores entre 0 y 1 a las categorías de horas trabajadas permite al modelo de clasificación entender la relación progresiva entre cada rango de horas trabajadas. A continuación, se justifica cada valor asignado a cada categoría:

'0': Valor 0.0. Representa la ausencia de horas trabajadas. Se asigna 0.0 ya que es el menor valor posible en el rango, indicando que no se trabaja nada en absoluto.

'Menos de 10 horas': Valor 0.25. Este valor representa una baja cantidad de horas trabajadas, pero superior a cero. Se asigna 0.25 para indicar que es un nivel bajo de trabajo, pero no nulo, marcando una ligera diferencia respecto al valor '0'.

'Entre 11 y 20 horas': Valor 0.5. Representa una cantidad intermedia de horas trabajadas. Se asigna 0.5 porque es aproximadamente la mitad del rango, indicando una carga de trabajo moderada en comparación con las otras categorías.

'Entre 21 y 30 horas': Valor 0.75. Este valor indica un nivel alto de horas trabajadas, pero no el máximo. Se asigna 0.75 porque se encuentra en el tramo superior, indicando un aumento significativo respecto a los rangos anteriores.

'Más de 30 horas': Valor 1.0. Representa el mayor rango de horas trabajadas. Se asigna 1.0 ya que es el valor máximo posible en la escala, indicando la carga de trabajo más alta y sirviendo como referencia para los valores anteriores.


In [ ]:
# Crear un diccionario para las horas trabajadas
horas_trabajo = {
    'Menos de 10 horas': 0.25,
    'Entre 11 y 20 horas': 0.5,
    'Entre 21 y 30 horas': 0.75,
    'Más de 30 horas': 1,
    '0': 0  # Si hay valores como '0', puedes tratarlos de manera especial
}

# Aplicar el diccionario
df['HORAS_TRABAJO_NORMALIZADO'] = df['ESTU_HORASSEMANATRABAJA'].map(horas_trabajo)
df = df.drop('ESTU_HORASSEMANATRABAJA', axis=1)

# Justificación
Convertir los valores "Sí" y "No" en 0 y 1 de la columna "**FAMI_TIENEINTERNET**" y "**ESTU_PAGOMATRICULAPROPIO**".

Muchos algoritmos de machine learning requieren que las variables sean numéricas. Al convertir "Sí" en 1 y "No" en 0, se transforma de una variable categórica a una numérica, así el modelo puede procesar de manera efectiva estos datos.

In [ ]:
# Eliminar espacios en blanco y convertir a minúsculas
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].str.strip().str.lower()
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].str.strip().str.lower()

# Convertir 'si' a 1 y 'no' a 0
df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].map({'si': 1, 'no': 0})
df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].map({'si': 1, 'no': 0})

# Justificación

Para la columna "**FAMI_ESTRATOVIVIENDA**" se utilizó::
- **Estrato 1**: Se convierte en 0 porque es el más bajo.

- **Estrato 6**: Se convierte en 1 porque es el más alto.

- **Estratos intermedios (2, 3, 4, 5)**: Se asignan valores de 0.2, 0.4, 0.6, 0.8 respectivamente, distribuyéndose de manera lineal entre 0 y 1.

In [ ]:
df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].replace({
    'Estrato 1': 0,
    'Estrato 2': 0.2,
    'Estrato 3': 0.4,
    'Estrato 4': 0.6,
    'Estrato 5': 0.8,
    'Estrato 6': 1,
    'Sin Estrato': -1
})

# Justificación
Para la columna "**RENDIMIENTO_GLOBAL**" se utilizó:

- **Bajo**: Se convierte en 0 porque es el más bajo.

- **Alto**: Se convierte en 1 porque es el más alto.

- **Medio-Bajo y Medio-Alto**: Se asignan valores de 0.33, y 0.66 respectivamente, distribuyéndose de manera lineal entre 0 y 1.

In [ ]:
# Crear un diccionario para el mapeo de rendimiento en valores enteros
rendimiento_discreto = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}

# Aplicar el diccionario
df['RENDIMIENTO_GLOBAL_NORMALIZADO'] = df['RENDIMIENTO_GLOBAL'].map(rendimiento_discreto)
df = df.drop('RENDIMIENTO_GLOBAL', axis=1)

# Justificación
Para normalizar las columnas "**FAMI_EDUCACIONPADRE**" y "**FAMI_EDUCACIONMADRE**" en una escala, se les asigna valores numéricos que reflejen el nivel de educación, desde el nivel más bajo "**Ninguno**" hasta el más alto "**Postgrado**".

- **Ninguno**: 0

- **Primaria incompleta**: 0.1

- **Primaria completa**: 0.2

- **Secundaria (Bachillerato) incompleta**: 0.3

- **Secundaria (Bachillerato) completa**: 0.4

- **Técnica o tecnológica incompleta**: 0.5

- **Técnica o tecnológica completa**: 0.6

- **Educación profesional incompleta**: 0.7

- **Educación profesional completa**: 0.8

- **Postgrado**: 1

Como no se sabe que valor numérico tomará **No sabe/No aplica** se asigna la media de esta manera:

- **No sabe**: Se asigna una media de 0.5

- **No aplica**: Se asigna una media de 0.5

In [ ]:
educacion_padre_normalizada = {
    'Ninguno': 0,
    'Primaria incompleta': 0.1,
    'Primaria completa': 0.2,
    'Secundaria (Bachillerato) incompleta': 0.3,
    'Secundaria (Bachillerato) completa': 0.4,
    'Técnica o tecnológica incompleta': 0.5,
    'Técnica o tecnológica completa': 0.6,
    'Educación profesional incompleta': 0.7,
    'Educación profesional completa': 0.8,
    'Postgrado': 1,
    'No sabe': 0.5,  # Asignando un valor intermedio
    'No Aplica': 0.5  # También un valor intermedio
}

# Aplicar el diccionario
df['FAMI_EDUCACIONPADRE_NORMALIZADO'] = df['FAMI_EDUCACIONPADRE'].map(educacion_padre_normalizada)
df = df.drop('FAMI_EDUCACIONPADRE', axis=1)

In [ ]:
df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].str.strip().str.lower()

educacion_madre_normalizada = {
    'ninguno': 0,
    'primaria incompleta': 0.1,
    'primaria completa': 0.2,
    'secundaria (bachillerato) incompleta': 0.3,
    'secundaria (bachillerato) completa': 0.4,
    'técnica o tecnológica incompleta': 0.5,
    'técnica o tecnológica completa': 0.6,
    'educación profesional incompleta': 0.7,
    'educación profesional completa': 0.8,
    'postgrado': 1,
    'no sabe': 0.5,
    'no aplica': 0.5
}

# Aplicar el diccionario
df['FAMI_EDUCACIONMADRE_NORMALIZADO'] = df['FAMI_EDUCACIONMADRE'].map(educacion_madre_normalizada)
df = df.drop('FAMI_EDUCACIONMADRE', axis=1)

# Justificación
Se normalizan las columnas "**ESTU_PRGM_DEPARTAMENTO**" y "**ESTU_PRGM_ACADEMICO**" de la siguiente manera

Se ordenan alfabéticamente.

Luego se asigna un valor entre 0 y 1 basado en su posición en la lista ordenada, usando la fórmula
$$
\frac{i}{(len(departamentos) - 1)}
$$

Donde "**i**" es el índice del departamento en la lista ordenada.

$$
\frac{i}{(len(programa) - 1)}
$$

Donde "**i**" es el índice del programa académico en la lista ordenada.


In [ ]:
# Lista de departamentos
departamentos = df['ESTU_PRGM_DEPARTAMENTO'].unique()

# Crear un diccionario para normalizar
departamentos_normalizados = {dep: i / (len(departamentos) - 1) for i, dep in enumerate(sorted(departamentos))}

# Reemplazar los nombres de los departamentos por sus valores normalizados
df['DEPARTAMENTO_NORMALIZADO'] = df['ESTU_PRGM_DEPARTAMENTO'].map(departamentos_normalizados).round(2)
df = df.drop('ESTU_PRGM_DEPARTAMENTO', axis=1)


In [ ]:
# Lista de programas
programas = df['ESTU_PRGM_ACADEMICO'].unique()

# Crear un diccionario para normalizar
programas_normalizados = {dep: i / (len(programas) - 1) for i, dep in enumerate(sorted(programas))}

# Reemplazar los nombres de los departamentos por sus valores normalizados
df['PRGM_ACADEMICO_NORMALIZADO'] = df['ESTU_PRGM_ACADEMICO'].map(programas_normalizados).round(2)
df = df.drop('ESTU_PRGM_ACADEMICO', axis=1)


In [ ]:
df

,ID,PERIODO,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,VALOR_MATRICULA_NORMALIZADO,HORAS_TRABAJO_NORMALIZADO,RENDIMIENTO_GLOBAL_NORMALIZADO,FAMI_EDUCACIONPADRE_NORMALIZADO,FAMI_EDUCACIONMADRE_NORMALIZADO,DEPARTAMENTO_NORMALIZADO,PRGM_ACADEMICO_NORMALIZADO
0,904256,20212,0.4,1,0,1.00,0.25,2,0.5,1.0,0.13,0.32
1,645256,20212,0.4,0,0,0.50,0.00,0,0.6,0.5,0.10,0.26
2,308367,20203,0.4,1,0,0.50,1.00,0,0.4,0.4,0.13,0.86
3,470353,20195,0.6,1,0,0.75,0.00,3,0.5,0.4,0.87,0.01
4,989032,20212,0.4,1,0,0.50,0.75,1,0.2,0.2,0.03,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,0.2,1,1,0.00,0.50,2,0.4,0.3,0.57,0.15
692496,754213,20212,0.4,1,0,0.50,1.00,0,0.1,0.3,0.70,0.96
692497,504185,20183,0.4,1,1,0.25,0.25,1,0.4,0.3,0.13,0.10
692498,986620,20195,0.0,0,1,0.50,0.25,0,0.2,0.2,0.93,0.96


In [ ]:
X = df.drop('RENDIMIENTO_GLOBAL_NORMALIZADO', axis=1).values
Y = df['RENDIMIENTO_GLOBAL_NORMALIZADO'].values
print (X.shape , Y.shape)
#1

(692500, 11) (692500,)


In [ ]:
print(df['VALOR_MATRICULA_NORMALIZADO'].unique())

[1.   0.5  0.75 0.25 0.  ]


In [ ]:
print(df.iloc[5])

ID                                 659872.00
PERIODO                             20203.00
FAMI_ESTRATOVIVIENDA                    0.80
FAMI_TIENEINTERNET                      1.00
ESTU_PAGOMATRICULAPROPIO                0.00
VALOR_MATRICULA_NORMALIZADO             0.50
HORAS_TRABAJO_NORMALIZADO               0.25
RENDIMIENTO_GLOBAL_NORMALIZADO          2.00
FAMI_EDUCACIONPADRE_NORMALIZADO         0.80
FAMI_EDUCACIONMADRE_NORMALIZADO         0.40
DEPARTAMENTO_NORMALIZADO                0.03
PRGM_ACADEMICO_NORMALIZADO              0.86
Name: 5, dtype: float64


In [ ]:
X

array([[9.04256e+05, 2.02120e+04, 4.00000e-01, ..., 1.00000e+00,
        1.30000e-01, 3.20000e-01],
       [6.45256e+05, 2.02120e+04, 4.00000e-01, ..., 5.00000e-01,
        1.00000e-01, 2.60000e-01],
       [3.08367e+05, 2.02030e+04, 4.00000e-01, ..., 4.00000e-01,
        1.30000e-01, 8.60000e-01],
       ...,
       [5.04185e+05, 2.01830e+04, 4.00000e-01, ..., 3.00000e-01,
        1.30000e-01, 1.00000e-01],
       [9.86620e+05, 2.01950e+04, 0.00000e+00, ..., 2.00000e-01,
        9.30000e-01, 9.60000e-01],
       [9.33374e+05, 2.01950e+04, 6.00000e-01, ..., 6.00000e-01,
        3.00000e-02, 9.60000e-01]])

In [ ]:
Y

array([2, 0, 0, ..., 1, 0, 3])

Separacion de test y train: (Para probar codigos y disminuir tiempo de ejecucion)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify=Y,test_size=0.30)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(484750, 11)
(207750, 11)
(484750,)
(207750,)


Funcion para graficar matriz de confusion:

In [ ]:
  #NORMALIZAR
def calcularMatrizCF(matrix,titulo):
    a = (matrix[0,0] +  matrix[0,1])
    b = (matrix[1,0] +  matrix[1,1])
    matrix[0,0] =  matrix[0,0] / a
    matrix[0,1] =  matrix[0,1] / a
    matrix[1,0] =  matrix[1,0] / b
    matrix[1,1] =  matrix[1,1] / b
    #MATRIZ DE CONFUSION
    sns.heatmap(matrix,annot=True)
    plt.xlabel('Label Pred')
    plt.ylabel('Label True')
    plt.title(titulo)
    print(matrix)

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd

def gradient_boosting_tree(X, y, num_trees):
    # Convierte y a etiquetas categóricas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    skf = StratifiedKFold(n_splits=4)
    resultados = pd.DataFrame()
    conf_matrix_list_of_arrays = np.zeros((len(np.unique(y_encoded)), len(np.unique(y_encoded))))

    idx = 0
    for trees in num_trees:
        EficienciaTrain = []
        EficienciaVal = []
        accuracy = []
        Macc = []
        Mpre = []
        Mrec = []
        Mf1 = []

        print(f"Training with {trees} trees...")

        for train_index, test_index in skf.split(X, y_encoded):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y_encoded[train_index], y_encoded[test_index]

            # Ajustando los hiperparámetros del modelo
            model = GradientBoostingClassifier(
                n_estimators=trees,
                learning_rate=0.05,  # Ajusta este parámetro
                max_depth=5,  # Ajusta este parámetro
                subsample=0.8,
                random_state=0
            )
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)

            conf_matrix = confusion_matrix(y_test, y_pred)
            conf_matrix_list_of_arrays += conf_matrix

            EficienciaTrain.append(np.mean(y_pred_train.ravel() == y_train.ravel()))
            EficienciaVal.append(np.mean(y_pred.ravel() == y_test.ravel()))
            accuracy.append(model.score(X_test, y_test))
            Macc.append(accuracy_score(y_test, y_pred))
            Mpre.append(precision_score(y_test, y_pred, average="weighted"))
            Mrec.append(recall_score(y_test, y_pred, average="weighted"))
            Mf1.append(f1_score(y_test, y_pred, average="weighted"))

        # Resultados por número de árboles
        resultados.loc[idx, 'número de árboles'] = trees
        resultados.loc[idx, 'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx, 'desviación estándar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx, 'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx, 'Intervalo de confianza (prueba)'] = np.std(EficienciaVal)
        resultados.loc[idx, 'accuracy real'] = np.mean(Macc)
        resultados.loc[idx, 'precision_score'] = np.mean(Mpre)
        resultados.loc[idx, 'recall_score'] = np.mean(Mrec)
        resultados.loc[idx, 'f1_score'] = np.mean(Mf1)
        idx += 1

    return resultados, conf_matrix_list_of_arrays


In [ ]:
resultados, conf_matrix = gradient_boosting_tree(X_train, y_train, [200])
print(resultados)

Training with 200 trees...
   número de árboles  eficiencia de entrenamiento  \
0              200.0                     0.425473   

   desviación estándar entrenamiento  eficiencia de prueba  \
0                           0.000883              0.413283   

   Intervalo de confianza (prueba)  accuracy real  precision_score  \
0                         0.001636       0.413283         0.393885   

   recall_score  f1_score  
0      0.413283  0.396697  


In [ ]:
resultados

,número de árboles,eficiencia de entrenamiento,desviación estándar entrenamiento,eficiencia de prueba,Intervalo de confianza (prueba),accuracy real,precision_score,recall_score,f1_score
0,200.0,0.425473,0.000883,0.413283,0.001636,0.413283,0.393885,0.413283,0.396697
